# CODE

In [2]:
# ============================================================
# @title INSTALL – Agnostic Agent (GitHub) + Infra Qwen3
# ============================================================
# 1. SUPER CLEANUP (Delete conflicting folders)
!rm -rf agnostic_agent  # <--- Deletes the lingering folder that causes ImportError
!rm -rf repo_agnostic   # <--- Clean start

# 2. CLONE REPO (Into 'repo_agnostic' to avoid any persistent name clash)
!git clone https://github.com/JacoboGGLeon/agnostic_agent.git repo_agnostic
!pip install -e repo_agnostic

# 3. FIX PATH (Tell Python where the package is)
import sys, os
pkg_path = os.path.abspath("repo_agnostic")
if pkg_path not in sys.path: sys.path.insert(0, pkg_path)

# 4. OTHER DEPENDENCIES
!pip -q install "vllm>=0.9.0" "huggingface_hub>=0.23.0" "openai==1.108.2"
!pip -q install langchain langchain-core langchain-openai langgraph
!pip -q install "git+https://github.com/whynpc9/langchain-qwq-vllm.git"
!pip -q install "pydantic>=2.7.0" "pyyaml>=6.0" streamlit sqlite-vec pandas numpy

print("✅ Agnostic Agent INSTALLED from GitHub!")
print("📂 Repo Path: repo_agnostic")
print("⚠️ IMPORTANT: If imports fail, go to Runtime -> Restart session, then run DOWNLOAD MODELS directly.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.4/948.4 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 MB 1.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.6/192.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 99.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00:00:0100:01
   ━

In [4]:
!ls -la

total 16
drwxr-xr-x 1 root root 4096 Dec  9 14:41 .
drwxr-xr-x 1 root root 4096 Feb  6 19:38 ..
drwxr-xr-x 4 root root 4096 Dec  9 14:41 .config
drwxr-xr-x 1 root root 4096 Dec  9 14:42 sample_data


# DEFINE vLLM

In [ ]:
# TRY

#@title DOWNLOAD MODELS – LLM, Embedding, Reranker (Qwen3)
from agnostic_agent import prepare_qwen_models

# IDs de modelos (puedes ajustarlos)
LLM_MODEL_ID = "Qwen/Qwen3-0.6B"  #@param ["Qwen/Qwen3-0.6B", "Qwen/Qwen3-0.6B-Base", "Qwen/Qwen3-4B", "Qwen/Qwen3-4B-Instruct-2507", "Qwen/Qwen3-4B-Thinking-2507"]

EMB_MODEL_ID = "Qwen/Qwen3-Embedding-0.6B"  #@param ["Qwen/Qwen3-Embedding-0.6B"]

RERANK_MODEL_ID = "Qwen/Qwen3-Reranker-0.6B"  #@param ["Qwen/Qwen3-Reranker-0.6B"]

print("⬇️ Descargando / preparando modelos Qwen3 (prepare_qwen_models)...")
model_paths = prepare_qwen_models(
    llm_model_id=LLM_MODEL_ID,
    emb_model_id=EMB_MODEL_ID,
    rerank_model_id=RERANK_MODEL_ID,
    base_dir="LM_MODEL",
)
print("\n✅ Modelos listos en disco:")
print("  LLM   :", model_paths.llm_dir)
print("  EMB   :", model_paths.emb_dir)
print("  RERANK:", model_paths.rerank_dir)

import os
os.environ["LLM_MODEL_ID"]   = LLM_MODEL_ID
os.environ["EMB_MODEL_ID"]   = EMB_MODEL_ID
os.environ["RERANK_MODEL_ID"] = RERANK_MODEL_ID


In [ ]:
#@title LIMPIEZA LIGERA DE GPU (opcional)
import gc
try:
    import torch
    has_torch = True
except ImportError:
    has_torch = False

print("🧹 Limpiando referencias de Python...")
gc.collect()
if has_torch and torch.cuda.is_available():
    print("🧠 Vaciando caché CUDA...")
    torch.cuda.empty_cache()
else:
    print("ℹ️ Torch CUDA no disponible o sin GPU visible.")


In [ ]:
#@title DEPLOY vLLM SERVER – SOLO LLM (chat) · L4/T4-friendly
import transformers
from agnostic_agent import VllmConfig, start_qwen_vllm_servers

print("Transformers version:", transformers.__version__)

VLLM_HOST = os.getenv("VLLM_HOST", "127.0.0.1")
LLM_PORT = 8000
EMB_PORT = 8001
RERANK_PORT = 8002
VLLM_MODE = "POWER"  # "FAST", "MEDIUM", "POWER", "LIMIT"

print(f"\n🚀 Lanzando servidor vLLM (start_qwen_vllm_servers)...")
print(f"🔧 VLLM_MODE = {VLLM_MODE}")

if VLLM_MODE == "FAST":
    vllm_cfg = VllmConfig(
        host=VLLM_HOST,
        llm_port=LLM_PORT,
        emb_port=EMB_PORT,
        rerank_port=RERANK_PORT,
        llm_gpu_util=0.40,
        llm_max_len=2048,
        llm_max_num_seqs=4,
        emb_gpu_util=0.30,
        emb_max_len=512,
        emb_max_num_seqs=4,
        rerank_gpu_util=0.30,
        rerank_max_len=512,
        rerank_max_num_seqs=4,
        enable_reasoning=True,
        tool_call_parser="hermes",
        reasoning_parser="qwen3",
        start_emb_server=False,
        start_rerank_server=False,
    )
elif VLLM_MODE == "MEDIUM":
    vllm_cfg = VllmConfig(
        host=VLLM_HOST,
        llm_port=LLM_PORT,
        emb_port=EMB_PORT,
        rerank_port=RERANK_PORT,
        llm_gpu_util=0.55,
        llm_max_len=4096,
        llm_max_num_seqs=6,
        emb_gpu_util=0.30,
        emb_max_len=512,
        emb_max_num_seqs=4,
        rerank_gpu_util=0.30,
        rerank_max_len=512,
        rerank_max_num_seqs=4,
        enable_reasoning=True,
        tool_call_parser="hermes",
        reasoning_parser="qwen3",
        start_emb_server=False,
        start_rerank_server=False,
    )
elif VLLM_MODE == "POWER":
    # Modo POWER “cargado” para L4 + Qwen3-0.6B:
    # - Más contexto (16k tokens)
    # - Menos concurrencia (4 secuencias)
    # - GPU util ~0.8 para exprimir la L4 sin matarla
    vllm_cfg = VllmConfig(
        # config base
        host=VLLM_HOST,
        llm_port=LLM_PORT,
        emb_port=EMB_PORT,
        rerank_port=RERANK_PORT,
        # language model
        llm_gpu_util=0.90,   # antes 0.70
        llm_max_len=32768,   # antes 8192  ← clave
        llm_max_num_seqs=1,  # antes 8     ← menos batch, más contexto
        # embedding model
        emb_gpu_util=0.05,
        emb_max_len=1024,
        emb_max_num_seqs=1,
        # reranker model
        rerank_gpu_util=0.05,
        rerank_max_len=1024,
        rerank_max_num_seqs=1,
        # config extra
        enable_reasoning=True,
        tool_call_parser="hermes",
        reasoning_parser="qwen3",
        start_emb_server=False,
        start_rerank_server=False,
    )

elif "LIMIT":
    vllm_cfg = VllmConfig(
        host=VLLM_HOST,
        llm_port=LLM_PORT,
        emb_port=EMB_PORT,
        rerank_port=RERANK_PORT,
        llm_gpu_util=0.85,    # un poco más agresivo
        llm_max_len=32768,    # 🔥 32k tokens de contexto
        llm_max_num_seqs=2,   # menos batch, más contexto por secuencia
        emb_gpu_util=0.35,
        emb_max_len=1024,
        emb_max_num_seqs=4,
        rerank_gpu_util=0.35,
        rerank_max_len=1024,
        rerank_max_num_seqs=4,
        enable_reasoning=True, # True | False
        tool_call_parser="hermes",
        reasoning_parser="qwen3",    # "qwen3" | none
        start_emb_server=True,
        start_rerank_server=False,
    )

endpoints, servers = start_qwen_vllm_servers(
    model_paths=model_paths,
    config=vllm_cfg,
    set_env=True,
)

print("\n✅ Servidor vLLM (LLM) listo.")


# RUN EXPERIMENT

In [ ]:
#@title streamlit_app.py
%%writefile streamlit_app.py
from __future__ import annotations

import os
import json
import html
from typing import Any, Dict, Optional, List, Tuple

import streamlit as st
from agnostic_agent.agent import Agent

# -----------------------------
# Page
# -----------------------------
st.set_page_config(
    page_title="Agnostic Agent · Chat Studio (Inspector)",
    page_icon="🧪",
    layout="wide",
)

# -----------------------------
# Hide Streamlit chrome (dark cintillo)
# -----------------------------
st.markdown("""
<style>
[data-testid="stHeader"] { display: none !important; }
[data-testid="stToolbar"] { display: none !important; }
[data-testid="stDecoration"] { display: none !important; }
#MainMenu { visibility: hidden !important; }
footer { visibility: hidden !important; }
.block-container { padding-top: 1rem !important; }
</style>
""", unsafe_allow_html=True)

# -----------------------------
# CSS (Studio + Inspector layout)
# -----------------------------
st.markdown(
    """
<style>
:root{
  --bg: #0b1020;
  --panel: rgba(255,255,255,.06);
  --panel2: rgba(255,255,255,.08);
  --border: rgba(255,255,255,.10);
  --text: rgba(255,255,255,.92);
  --muted: rgba(255,255,255,.65);
  --accent: #7c5cff;
  --good: #2dd4bf;

  --r: 18px;
  --r2: 14px;
  --shadow: 0 12px 35px rgba(0,0,0,.35);
  --mono: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, "Liberation Mono", "Courier New", monospace;
}

.stApp{
  background:
    radial-gradient(1200px 500px at 10% -10%, rgba(124,92,255,.35), transparent 60%),
    radial-gradient(900px 500px at 90% 0%, rgba(45,212,191,.18), transparent 60%),
    linear-gradient(180deg, var(--bg), #070a14 60%, #050712);
  color: var(--text);
}

.block-container{ padding-top: 1.0rem; padding-bottom: 1.6rem; }

section[data-testid="stSidebar"]{
  background: rgba(0,0,0,.18);
  border-right: 1px solid rgba(255,255,255,.06);
}

.topbar{
  display:flex; align-items:center; justify-content:space-between;
  gap:12px;
  padding: 12px 14px;
  border-radius: var(--r);
  background: linear-gradient(180deg, rgba(255,255,255,.08), rgba(255,255,255,.05));
  border: 1px solid var(--border);
  box-shadow: var(--shadow);
  margin-bottom: 10px;
}
.brand{display:flex; align-items:center; gap:10px;}
.logo{
  width: 38px; height: 38px; border-radius: 12px;
  display:flex; align-items:center; justify-content:center;
  background: linear-gradient(135deg, rgba(124,92,255,.9), rgba(45,212,191,.6));
  box-shadow: 0 10px 25px rgba(124,92,255,.22);
  font-size: 18px;
}
.title{font-size: 15px; font-weight: 800; line-height: 1.1;}
.subtitle{font-size: 12px; color: var(--muted);}

.badges{display:flex; flex-wrap:wrap; gap:8px; justify-content:flex-end;}
.badge{
  font-size: 12px;
  padding: 6px 10px;
  border-radius: 999px;
  border: 1px solid var(--border);
  background: rgba(255,255,255,.06);
  color: var(--text);
}
.badge.accent{ border-color: rgba(124,92,255,.45); }
.badge.good{ border-color: rgba(45,212,191,.45); }

.card{
  border-radius: var(--r);
  border: 1px solid var(--border);
  background: rgba(255,255,255,.06);
  box-shadow: 0 10px 30px rgba(0,0,0,.28);
  overflow: hidden;
  margin-top: 6px;
}
.card .card-h{
  display:flex; align-items:center; justify-content:space-between;
  gap:10px;
  padding: 10px 12px;
  background: rgba(255,255,255,.05);
  border-bottom: 1px solid var(--border);
  font-weight: 800;
  font-size: 13px;
}
.card .card-h .hint{
  font-weight: 500; font-size: 11px; color: var(--muted);
}
.card .card-b{
  padding: 12px 12px 10px 12px;
  font-size: 14px;
  color: var(--text);
}

/* Code-like block inside cards (for Thinking) */
.codebox{
  margin-top: 8px;
  padding: 10px 12px;
  border-radius: 14px;
  border: 1px solid rgba(255,255,255,.10);
  background: rgba(0,0,0,.28);
  font-family: var(--mono);
  font-size: 12px;
  line-height: 1.45;
  white-space: pre-wrap;
  word-break: break-word;
  color: rgba(255,255,255,.92);
}

/* User bubble */
.bubble-user{
  padding: 10px 12px;
  border-radius: 16px;
  border: 1px solid rgba(124,92,255,.35);
  background: linear-gradient(180deg, rgba(124,92,255,.22), rgba(255,255,255,.05));
  box-shadow: 0 8px 24px rgba(0,0,0,.25);
}

/* Inspector wrapper */
.inspector{
  border-radius: var(--r);
  border: 1px solid var(--border);
  background: rgba(255,255,255,.05);
  box-shadow: var(--shadow);
  padding: 12px;
}
.inspector h3{ margin: 0 0 6px 0; }

/* Expanders */
[data-testid="stExpander"]{
  border-radius: var(--r);
  border: 1px solid var(--border);
  background: rgba(255,255,255,.04);
  overflow:hidden;
}

/* Chat spacing */
[data-testid="stChatMessage"]{
  padding-top: 0.25rem;
  padding-bottom: 0.25rem;
}
</style>
""",
    unsafe_allow_html=True,
)

# -----------------------------
# Session state
# -----------------------------
if "agent" not in st.session_state:
    st.session_state.agent = None
if "agent_mode" not in st.session_state:
    st.session_state.agent_mode = "tools_strict"
if "messages" not in st.session_state:
    st.session_state.messages = []
if "msg_counter" not in st.session_state:
    st.session_state.msg_counter = 0
if "selected_msg_id" not in st.session_state:
    st.session_state.selected_msg_id = None
if "export_json" not in st.session_state:
    st.session_state.export_json = None

# -----------------------------
# Sidebar controls
# -----------------------------
with st.sidebar:
    st.markdown("## 🧪 Chat Studio")
    agent_mode = st.selectbox(
        "Policy mode",
        ["tools_strict", "free_policies"],
        index=0 if st.session_state.agent_mode == "tools_strict" else 1,
    )

    st.markdown("### 🧭 Inspector")
    show_thinking_tab = st.checkbox("🧠 Thinking", value=True)
    show_deep_tab = st.checkbox("🧠 Deep", value=True)
    show_dev_tab = st.checkbox("🔍 Dev", value=True)

    st.markdown("### 🧹 Acciones")
    cA, cB = st.columns(2)
    with cA:
        if st.button("🗑️ Limpiar", use_container_width=True):
            st.session_state.messages = []
            st.session_state.agent = None
            st.session_state.selected_msg_id = None
            st.toast("Chat reiniciado.", icon="🧹")
            st.rerun()
    with cB:
        if st.button("⬇️ Export", use_container_width=True):
            export = {"agent_mode": st.session_state.agent_mode, "messages": st.session_state.messages}
            st.session_state.export_json = json.dumps(export, ensure_ascii=False, indent=2)
            st.toast("Transcript listo.", icon="⬇️")

    if isinstance(st.session_state.export_json, str):
        st.download_button(
            "Descargar transcript.json",
            data=st.session_state.export_json,
            file_name="transcript.json",
            mime="application/json",
            use_container_width=True,
        )

# Mode change => reset agent (history stays)
if st.session_state.agent_mode != agent_mode:
    st.session_state.agent_mode = agent_mode
    st.session_state.agent = None
    st.toast(f"Modo: {agent_mode}", icon="🧭")

# -----------------------------
# Helpers
# -----------------------------
def next_id() -> int:
    st.session_state.msg_counter += 1
    return st.session_state.msg_counter

def get_or_init_agent(mode: str) -> Agent:
    if st.session_state.agent is None:
        os.environ["PLANNER_POLICY_MODE"] = mode
        os.environ["AGENT_POLICY_MODE"] = mode
        with st.spinner(f"Inicializando agente (mode={mode})…"):
            try:
                st.session_state.agent = Agent.init(policy_mode=mode)
            except TypeError:
                st.session_state.agent = Agent.init()
    return st.session_state.agent

def normalize_output(raw: Any) -> Dict[str, Any]:
    if raw is None:
        return {}
    if hasattr(raw, "model_dump"):
        try:
            return raw.model_dump()
        except TypeError:
            pass
    if isinstance(raw, dict):
        return raw
    return {"user_out": str(raw)}

def as_text(v: Any) -> str:
    if v is None:
        return ""
    if isinstance(v, str):
        return v.strip()
    if isinstance(v, dict):
        for k in ("final_answer", "text", "content", "answer", "user_out"):
            vv = v.get(k)
            if isinstance(vv, str) and vv.strip():
                return vv.strip()
        return ""
    return str(v).strip()

def strip_user_prefix(text: str) -> str:
    if not text:
        return ""
    t = text.strip()
    prefixes = [
        "Respuesta final (modo usuario):",
        "**Respuesta final (modo usuario):**",
        "RESPUESTA FINAL (modo usuario):",
    ]
    for p in prefixes:
        if t.startswith(p):
            t = t[len(p):].strip()
    return t

def get_raw_state(out: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    if not isinstance(out, dict):
        return None
    if isinstance(out.get("messages"), list):
        return out
    rs = out.get("raw_state")
    if isinstance(rs, dict):
        return rs
    dev = out.get("dev_out")
    if isinstance(dev, dict) and isinstance(dev.get("raw_state"), dict):
        return dev["raw_state"]
    return None

def extract_thinking(raw_state: Optional[Dict[str, Any]]) -> str:
    if not isinstance(raw_state, dict):
        return ""
    msgs = raw_state.get("messages")
    if not isinstance(msgs, list):
        return ""
    for m in reversed(msgs):
        if not isinstance(m, dict):
            continue
        if m.get("type") != "ai":
            continue
        ak = m.get("additional_kwargs") or {}
        if isinstance(ak, dict) and ak.get("pipeline_internal"):
            continue
        thinking = ak.get("reasoning_content") or ak.get("reasoning") or ak.get("thoughts") or ""
        return thinking.strip() if isinstance(thinking, str) else ""
    return ""

def extract_summary_deep(raw_state: Optional[Dict[str, Any]], deep_out_text: str) -> str:
    if deep_out_text:
        return deep_out_text
    if not isinstance(raw_state, dict):
        return ""
    summary = raw_state.get("summary") or raw_state.get("pipeline_summary")
    if not isinstance(summary, dict):
        return ""
    parts = []
    for k in ["analyzer", "planner", "executor", "catcher", "summarizer", "final_answer"]:
        v = summary.get(k, "")
        if isinstance(v, str) and v.strip():
            parts.append(f"**{k.upper()}**\n\n{v.strip()}")
    return "\n\n---\n\n".join(parts) if parts else ""

def extract_tool_runs(out: Dict[str, Any], raw_state: Optional[Dict[str, Any]]) -> List[Dict[str, Any]]:
    if isinstance(raw_state, dict):
        tr = raw_state.get("tool_runs")
        if isinstance(tr, list):
            return tr
    tr2 = out.get("tool_runs")
    if isinstance(tr2, list):
        return tr2
    dev = out.get("dev_out")
    if isinstance(dev, dict) and isinstance(dev.get("tool_runs"), list):
        return dev["tool_runs"]
    return []

def assistant_messages() -> List[Dict[str, Any]]:
    return [m for m in st.session_state.messages if m.get("role") == "assistant"]

def find_message_by_id(msg_id: Optional[int]) -> Optional[Dict[str, Any]]:
    if msg_id is None:
        return None
    for m in st.session_state.messages:
        if m.get("id") == msg_id:
            return m
    return None

def default_selected_id() -> Optional[int]:
    a = assistant_messages()
    return a[-1]["id"] if a else None

def card_md(title: str, body_md: str, icon: str = "⬛", hint: str = "") -> None:
    body_md = body_md or "_(vacío)_"
    hint_html = f'<span class="hint">{html.escape(hint)}</span>' if hint else ""
    # NOTE: body_md here is treated as plain HTML content; for Deep this is OK.
    # For Thinking we use code-card below so it looks like "markdown blocks".
    st.markdown(
        f"""
<div class="card">
  <div class="card-h">
    <div>{icon} {html.escape(title)}</div>
    {hint_html}
  </div>
  <div class="card-b">{body_md}</div>
</div>
""",
        unsafe_allow_html=True,
    )

def card_code(title: str, code_text: str, icon: str = "🧠", hint: str = "reasoning_content") -> None:
    safe = html.escape(code_text or "")
    hint_html = f'<span class="hint">{html.escape(hint)}</span>' if hint else ""
    content = safe if safe.strip() else html.escape("_(no viene thinking en este turno)_")
    st.markdown(
        f"""
<div class="card">
  <div class="card-h">
    <div>{icon} {html.escape(title)}</div>
    {hint_html}
  </div>
  <div class="card-b">
    <div class="codebox">{content}</div>
  </div>
</div>
""",
        unsafe_allow_html=True,
    )

def render_tool_runs(tool_runs: List[Dict[str, Any]]) -> None:
    if not tool_runs:
        st.markdown("_(No se ejecutaron tools en este turno.)_")
        return
    st.markdown("#### 🛠 Tools ejecutadas")
    for i, tr in enumerate(tool_runs, start=1):
        if not isinstance(tr, dict):
            st.markdown(f"**{i}. tool_{i}**")
            st.code(str(tr))
            continue
        name = tr.get("name", f"tool_{i}")
        args = tr.get("args", {})
        output = tr.get("output", "")
        st.markdown(f"**{i}. {name}**")
        if args:
            st.code(args, language="json")
        if output != "":
            st.markdown("**Salida:**")
            st.code(str(output))

# If nothing selected yet, default to last assistant
if st.session_state.selected_msg_id is None:
    st.session_state.selected_msg_id = default_selected_id()

# -----------------------------
# Top bar
# -----------------------------
mode_badge = "tools_strict" if st.session_state.agent_mode == "tools_strict" else "free_policies"
mode_class = "accent" if st.session_state.agent_mode == "tools_strict" else "good"

st.markdown(
    f"""
<div class="topbar">
  <div class="brand">
    <div class="logo">🧪</div>
    <div>
      <div class="title">Agnostic Agent · Chat Studio</div>
      <div class="subtitle">Feed limpio + Inspector dinámico (thinking/deep/dev)</div>
    </div>
  </div>
  <div class="badges">
    <span class="badge {mode_class}">🧭 {mode_badge}</span>
    <span class="badge">🔎 inspector: on</span>
  </div>
</div>
""",
    unsafe_allow_html=True,
)

# -----------------------------
# Main layout: Feed | Inspector
# -----------------------------
feed_col, insp_col = st.columns([2.2, 1.0], gap="large")

# -------- FEED (left) --------
with feed_col:
    for msg in st.session_state.messages:
        role = msg.get("role", "user")

        if role == "user":
            with st.chat_message("user"):
                st.markdown(f'<div class="bubble-user">{html.escape(msg.get("content",""))}</div>', unsafe_allow_html=True)

        elif role == "assistant":
            out = msg.get("out") or {}
            answer = strip_user_prefix(as_text(out.get("user_out")))
            raw_state = get_raw_state(out)
            tool_runs = extract_tool_runs(out, raw_state)

            with st.chat_message("assistant"):
                # Pretty answer only
                card_md(
                    title="Respuesta",
                    body_md=html.escape(answer or "_(sin respuesta)_").replace("\n", "<br>"),
                    icon="👤",
                    hint=f"id={msg.get('id')}",
                )

                c1, c2, c3 = st.columns([1.2, 1.0, 0.8])
                with c1:
                    st.caption(f"🛠 tools: {len(tool_runs)}")
                with c2:
                    st.caption("📎 Inspector →")
                with c3:
                    if st.button("🔎 Inspect", key=f"inspect_{msg.get('id')}", use_container_width=True):
                        st.session_state.selected_msg_id = msg.get("id")
                        st.toast(f"Inspector → id={msg.get('id')}", icon="🔎")
                        st.rerun()

# -------- INSPECTOR (right) --------
with insp_col:
    st.markdown('<div class="inspector">', unsafe_allow_html=True)
    st.markdown("### 🔎 Inspector")

    a_msgs = assistant_messages()
    if not a_msgs:
        st.info("Aún no hay respuestas del agente. Escribe algo para empezar.")
        st.markdown("</div>", unsafe_allow_html=True)
    else:
        ids = [m["id"] for m in a_msgs]

        def label(mid: int) -> str:
            m = find_message_by_id(mid) or {}
            out = m.get("out") or {}
            text = strip_user_prefix(as_text(out.get("user_out"))).replace("\n", " ").strip()
            text = (text[:60] + "…") if len(text) > 60 else text
            return f"id={mid} · {text or '(sin texto)'}"

        if st.session_state.selected_msg_id not in ids:
            st.session_state.selected_msg_id = ids[-1]

        sel = st.selectbox(
            "Mensaje seleccionado",
            options=ids,
            index=ids.index(st.session_state.selected_msg_id),
            format_func=label,
            key="inspector_selectbox",
        )
        st.session_state.selected_msg_id = sel

        m = find_message_by_id(st.session_state.selected_msg_id) or {}
        out = m.get("out") or {}
        raw_state = get_raw_state(out)

        thinking = extract_thinking(raw_state)
        deep_txt = extract_summary_deep(raw_state, as_text(out.get("deep_out")))
        tool_runs = extract_tool_runs(out, raw_state)

        tab_specs: List[Tuple[str, str]] = []
        if show_thinking_tab:
            tab_specs.append(("🧠 Thinking", "thinking"))
        if show_deep_tab:
            tab_specs.append(("🧠 Deep", "deep"))
        if show_dev_tab:
            tab_specs.append(("🔍 Dev", "dev"))

        tabs = st.tabs([t[0] for t in tab_specs])

        for (tab_title, tab_key), tab in zip(tab_specs, tabs):
            with tab:
                if tab_key == "thinking":
                    # ✅ Now it looks like a proper "markdown code block" card (monospace, wrapped, bordered)
                    card_code("Pensamiento (thinking)", thinking, icon="🧠", hint="reasoning_content")

                elif tab_key == "deep":
                    # Keep as-is (you said you like it)
                    # We keep newlines readable and let emphasis render in markdown-ish style
                    body = deep_txt or "_(vacío)_"
                    # Render markdown emphasis by replacing to HTML-ish layout; deep usually ok as plain text too
                    body_html = html.escape(body).replace("\n", "<br>")
                    card_md("Vista profunda (deep_out / summary)", body_html, icon="🧠", hint="pipeline")

                elif tab_key == "dev":
                    # ✅ Dev: tools like before + raw_state collapsed
                    render_tool_runs(tool_runs)

                    with st.expander("🧬 raw_state (debug)", expanded=False):
                        if isinstance(raw_state, dict) and raw_state:
                            st.json(raw_state)
                        else:
                            st.markdown("_(sin raw_state)_")

        st.markdown("</div>", unsafe_allow_html=True)

# -----------------------------
# Chat input (bottom)
# -----------------------------
prompt = st.chat_input("Escribe tu mensaje…")

if prompt:
    uid = next_id()
    st.session_state.messages.append({"id": uid, "role": "user", "content": prompt, "out": None})

    agent = get_or_init_agent(st.session_state.agent_mode)
    try:
        raw_out = agent.run_turn(prompt, policy_mode=st.session_state.agent_mode)
    except TypeError:
        raw_out = agent.run_turn(prompt)

    out = normalize_output(raw_out)
    aid = next_id()
    st.session_state.messages.append(
        {"id": aid, "role": "assistant", "content": strip_user_prefix(as_text(out.get("user_out"))), "out": out}
    )

    st.session_state.selected_msg_id = aid
    st.rerun()


In [ ]:
#@title .streamlit/config.toml
!mkdir -p .streamlit

In [ ]:
%%writefile .streamlit/config.toml
[theme]
base="dark"
primaryColor="#7c5cff"
backgroundColor="#070a14"
secondaryBackgroundColor="#10162b"
textColor="#e9e9f2"
font="sans serif"


In [ ]:
#@title Lanzar servicio Streamlit en Colab (window o iframe)
import subprocess, time, os

# ---- PARAMS (Colab UI) ----
PORT = 8501  #@param {type:"integer"}
MODE = "iframe"  #@param ["window", "iframe"]
IFRAME_HEIGHT = 800  #@param {type:"integer"}
IFRAME_WIDTH = "100%"  #@param {type:"string"}

# ---- Colab output (solo si existe) ----
try:
    from google.colab import output
    _IN_COLAB = True
except Exception:
    output = None
    _IN_COLAB = False

# ---- Mata procesos previos (opcional) ----
os.system('pkill -f "streamlit run streamlit_app.py" || true')
os.system('pkill -f "streamlit" || true')

# ---- Lanza Streamlit ----
cmd = [
    "streamlit", "run", "streamlit_app.py",
    "--server.address", "0.0.0.0",
    "--server.port", str(PORT),
    "--server.headless", "true",
    "--server.enableCORS", "false",
    "--server.enableXsrfProtection", "false",
]

streamlit_process = subprocess.Popen(cmd)
time.sleep(3)

# ---- Exponer ----
if _IN_COLAB and output is not None:
    if MODE == "iframe":
        output.serve_kernel_port_as_iframe(
            PORT,
            width=IFRAME_WIDTH,
            height=IFRAME_HEIGHT,
        )
    else:
        output.serve_kernel_port_as_window(
            PORT,
            anchor_text="🔗 Abrir app Streamlit en una nueva pestaña"
        )
else:
    print(f"Streamlit corriendo en http://127.0.0.1:{PORT} (no-Colab).")
